<b>Exercise 4: Effects of neuron morphology.</b>
<br><br>
We continue with the model of exercise 2 (HH mechanism present in the soma, passive leak currents in the dendrites).

In [1]:
#Import the python libraries
import matplotlib.pyplot as plt
plt.ion()
from neuron import h
import numpy
import time
from ipywidgets import widgets, fixed, Layout

In [2]:
%%html
<style type='text/css'>
.widget-inline-hbox .widget-label {
      max-width: 250px;
      min-width: 250px;
}
</style>

In [3]:
#Run NEURON with Python interface
def runneuron(isadaptive=True,amp=0.12,L0=200,L1=100,L2=160,d0=5,d1=2,d2=2,glsoma=0.03,gldend=0.03,Ra=100,cm=1.0):
    my_start_time = time.time()
        
    h.load_file("stdlib.hoc")                        #A default NEURON library
    h.load_file("stdrun.hoc")                        #A default NEURON library

    h('objref cvode')                                #Define an object cvode
    h('cvode = new CVode()')                         #Make cvode a time step integrator object
    h('cvode.active('+str(isadaptive*1)+')')         #Set the variable time step integration on/off

    h('create soma, dend[3]')                        #Create soma and three dendritic sections

    h.dend[0].connect(h.soma,1,0)                    #Connect the sections to each other
    h.dend[1].connect(h.soma,1,0)
    h.dend[2].connect(h.dend[0],1,0)
    stimtextdict = {0: h.soma, 1: h.dend[0], 2: h.dend[1], 3: h.dend[2]}
    stimnamedict = {0: 'soma', 1: 'dend[0]', 2: 'dend[1]', 3: 'dend[2]'}

    h('access soma')                                 #Make soma the currently accessed section

    for sec in [h.dend[0], h.dend[1], h.dend[2]]:
        sec.insert('pas')                            #Insert passive membrane mechanism to all sections
        sec.nseg = 5                                 #and use 5 sub-compartments for each section
    h.soma.insert('hh')                              #Insert the Hodgkin-Huxley mechanism (includes leak) to soma
    h.soma.nseg = 5                                  

    #Set the passive parameters for each section:
    h.soma.Ra = Ra; h.soma.cm = cm; h.soma.diam = 10; h.soma.L = 20; h.soma.gl_hh = 0.001*glsoma
    h.dend[0].Ra = Ra; h.dend[0].cm = cm; h.dend[0].diam = d0; h.dend[0].L = L0; h.dend[0].e_pas = -54.3; h.dend[0].g_pas = 0.001*gldend
    h.dend[1].Ra = Ra; h.dend[1].cm = cm; h.dend[1].diam = d1; h.dend[1].L = L1; h.dend[1].e_pas = -54.3; h.dend[1].g_pas = 0.001*gldend
    h.dend[2].Ra = Ra; h.dend[2].cm = cm; h.dend[2].diam = d2; h.dend[2].L = L2; h.dend[2].e_pas = -54.3; h.dend[2].g_pas = 0.001*gldend

    h.dt = 0.025                                     #Set the time step to 0.025 ms
    h.tstop = 200                                    #Continue the simulation until 200 ms

    h('objref stim, Vrec')                           #Declare objects stim and Vrec

    #Make stim an IClamp object, stimulating at a given location of a given compartment
    h.stim = h.IClamp(0.5, sec = h.soma)
    
    h.stim.delay = 100                               #Stimulation starts at 100 ms
    h.stim.dur = 10                                  #and lasts 10 ms
    h.stim.amp = amp                                 #and has a given amplitude (default 0.075 nA)

    trec = h.Vector()                                #Save time instants in trec
    trec.record(h._ref_t)
    
    Vrecs = []                                       #Save segment-wise membrane potential values in Vrecs
    recxs = [0.1*x for x in range(0,10)]             #Record at ten locations along each compartment
    for icomp in range(0,4):                         #Iterate across compartments and recording locations
        for ix in range(0,len(recxs)):
            Vrecs.append(h.Vector())                 #Append a vector and record to the added vector
            Vrecs[-1].record(stimtextdict[icomp](recxs[ix])._ref_v, sec=stimtextdict[icomp])            

    h.init()                                         #Initialize state variables
    h.run()                                          #Run the simulations

    plt.close("all")
    f, axs = plt.subplots(2, 4, sharey=True)
    for j in range(0,4):
        axs[0,j].plot(trec,Vrecs[5+len(recxs)*j])
        axs[0,j].set_title(stimnamedict[j])
        axs[0,j].set_xlabel('$t$ (ms)')
        axs[0,j].set_xlim([0, 200])
        axs[0,j].set_ylim([-80, 40])
    axs[0,0].set_ylabel('$V_m(t)$ (mV)')
    axs[1,0].set_ylabel('max $V_m$ (mV)')
    for j in range(0,4):
        axs[1,j].plot(recxs, [max(Vrecs[i+len(recxs)*j]) for i in range(0,len(recxs))])
        axs[1,j].set_xlabel('x')
    axnew = f.add_axes([0.95,0.1,0.04,0.8])
    axnew.set_xlim([-0.2,0.2])
    axnew.set_ylim([-10,20+L0+L2+10])
    axnew.plot([0,0],[0,20],'b-',color='#6666FF',linewidth=0.6*10)
    axnew.plot([-0.05,-0.05],[20,L1+20],'c-',linewidth=0.6*d1)
    axnew.plot([0.05,0.05],[20,L0+20],'g-',linewidth=0.6*d0)
    axnew.plot([0.05,0.05],[L0+20,L2+L0+20],'y-',color='#AABB00',linewidth=0.6*d2)
    axnew.plot(0,20*0.5, 'kx')
    axnew.axis('off')
        
    plt.suptitle("Simulation run in "+str(time.time() - my_start_time)+" seconds.")
    plt.show()



In [4]:
slider = widgets.interact(runneuron, isadaptive=fixed(True),
                          amp=widgets.FloatSlider(min=0.0,max=0.3,step=0.01,value=0.12,description='Amplitude of square pulse current: amp',layout=Layout(width='50%'), continuous_update = False), 
                          L0=widgets.IntSlider(min=10,max=500,step=10,value=200,description='Length of dend[0]: L0',layout=Layout(width='50%'), continuous_update = False), 
                          L1=widgets.IntSlider(min=10,max=500,step=10,value=100,description='Length of dend[1]: L1',layout=Layout(width='50%'), continuous_update = False), 
                          L2=widgets.IntSlider(min=10,max=500,step=10,value=160,description='Length of dend[2]: L2',layout=Layout(width='50%'), continuous_update = False), 
                          d0=widgets.FloatSlider(min=0.5,max=10,step=0.1,value=5,description='Diameter of dend[0]: d0',layout=Layout(width='50%'), continuous_update = False), 
                          d1=widgets.FloatSlider(min=0.5,max=10,step=0.1,value=2,description='Diameter of dend[1]: d1',layout=Layout(width='50%'), continuous_update = False), 
                          d2=widgets.FloatSlider(min=0.5,max=10,step=0.1,value=2,description='Diameter of dend[2]: d2',layout=Layout(width='50%'), continuous_update = False), 
                          glsoma=widgets.FloatSlider(min=0.0,max=0.5,step=0.01,value=0.03,description='Leak conductance, soma: glsoma',layout=Layout(width='50%'), continuous_update = False), 
                          gldend=widgets.FloatSlider(min=0.0,max=0.5,step=0.01,value=0.03,description='Leak conductance, dend: gldend',layout=Layout(width='50%'), continuous_update = False), 
                          Ra=widgets.FloatSlider(min=10.0,max=1000.0,step=10,value=100,description='Axial resistance: Ra',layout=Layout(width='50%'), continuous_update = False),
                          cm=widgets.FloatSlider(min=0.1,max=10.0,step=0.1,value=1.0,description='Membrane capacitance: cm',layout=Layout(width='50%'), continuous_update = False))


interactive(children=(FloatSlider(value=0.12, continuous_update=False, description='Amplitude of square pulse …

#Question 1: Consider the cable equation 
$$\frac{1}{2\pi a}\frac{\partial}{\partial x}\left(\frac {\pi a^2}{R_a}\frac {\partial V}{\partial x}\right) = C_m \frac{\partial V}{\partial t} + I_{HH}$$
Which morphological parameters have largest effect on the electrotonic distance between soma and dend[2]? Test with the sliders how these affect the neuron behaviour. The first row of panels shows the membrane potential time course at the center of each compartment, while the second row shows the maximal (over time) membrane potential at each point $x\in[0,1]$ of each of the four compartments.

#Question 2a: Keeping $C_m$ at 1 $\mu\mathrm{F}/\mathrm{cm}^2$, make the dendrites as thick as the soma (diameter 10 $\mu$m) and maximum length (500 $\mu$m). Can you make the neuron spike? Why/why not?
#Question 2b: Vary $C_m$, and see if the situation changes. 